# Topic Modeling (LDA)
## Setup


In [1]:
!pip install --upgrade gensim

In [2]:
!pip install nltk
!pip install pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 21.1 MB/s eta 0:00:00


## import libraries

In [3]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
import numpy as np
import json
import glob

#gensim
import gensim
import gensim.corpora as copora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Data Prep

In [5]:
def laod_data(file):
  with open (file, "r", encoding="utf-8") as f:
    data = json.load(f)
  return (data)

def write_data(file, data):
  with open (file, "w", encoding="utf-8") as f:
    json.dump(data, f, indent=4)

In [6]:
stopwords = stopwords.words("english")
print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [10]:
data = laod_data("/content/open_answers_and_comments.json")["answers"]

print(data[0][0:90])

How to test if string exists in file with Bash?


In [11]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
  nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
  texts_out = []
  for text in texts:
    doc = nlp(text)
    new_text = []
    for token in doc:
      if token.pos_ in allowed_postags:
        new_text.append(token.lemma_)
    final = " ".join(new_text)
    texts_out.append(final)
  return(texts_out)

lemmatized_text = lemmatization(data)
print(lemmatized_text[0][0:90])

test string exist file


In [12]:
def gen_words(texts):
  final = []
  for text in texts:
    new = gensim.utils.simple_preprocess(text, deacc=True)
    final.append(new)
  return (final)

data_words = gen_words(lemmatized_text)

print(data_words[0][0:30])

['test', 'string', 'exist', 'file']


In [13]:
id2word = gensim.corpora.Dictionary(data_words)

corpus = []

for text in data_words:
  new = id2word.doc2bow(text) # bow -> bag of words
  corpus.append(new)

print(corpus[0][0:20]) #each tuple contains index of the word along with the frequency of that word.

[(0, 1), (1, 1), (2, 1), (3, 1)]


In [14]:
lda = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                    id2word=id2word,
                                    num_topics=10,
                                    random_state=100,
                                    update_every=1,
                                    chunksize=100,
                                    passes=10,
                                    alpha="auto")

## Visualization of Data

In [15]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda, corpus, id2word, mds="mmds", R=9)
vis

/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.394676  0.247155       1        1  18.692730
6      0.135851  0.438302       2        1  16.544697
3     -0.309050 -0.236964       3        1  12.371336
1     -0.231317  0.428792       4        1  11.788542
8      0.215566 -0.428632       5        1  11.144580
0     -0.223708  0.192233       6        1  10.585790
2     -0.121365 -0.444902       7        1   7.285241
4     -0.490007 -0.010458       8        1   5.381707
5      0.167656 -0.074100       9        1   3.174185
7      0.461697 -0.111427      10        1   3.031191, topic_info=         Term          Freq         Total Category  logprob  loglift
18        use  20690.000000  20690.000000  Default   9.0000   9.0000
145  function  14272.000000  14272.000000  Default   8.0000   8.0000
62      error   5078.000000   5078.000000  Default   7.0000   7.0000
14      check   7329.000000   7329.000000  Default   6.0000   6.0000
188     other   4887.000000   4887.000000  Default   5.0000   5.0000
..        ...           ...           ...      ...      ...      ...
252     super   1514.361455   1515.285632  Topic10  -2.9143   3.4956
251      hole   1476.285087   1477.209264  Topic10  -2.9397   3.4956
250  extremal   1476.284255   1477.208432  Topic10  -2.9397   3.4956
219    ground   1435.044268   1435.968450  Topic10  -2.9681   3.4956
129    become   1505.638233   3842.446326  Topic10  -2.9200   2.5593

[104 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
397       4  0.999646    access
328       8  0.999380  argument
556       4  0.999771     array
1892      6  0.999345       bar
129       1  0.260771    become
...     ...       ...       ...
383       8  0.999691     valid
13        4  0.999903     value
38        2  0.999857  variable
150       5  0.999624       way
554       5  0.999797      work

[99 rows x 3 columns], R=9, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 7, 4, 2, 9, 1, 3, 5, 6, 8])

In [16]:
print('\nPerplexity: ', lda.log_perplexity(corpus,total_docs=10000))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda, texts=data_words, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.993703884260169

Coherence Score:  0.4521789121122451
